In [12]:
import cProfile
import simulation

In [24]:
sim = simulation.Simulation(300, 100_000, 14, 0.6, 2)

In [25]:
sim.init_simulation()

In [26]:
sim.run_simulation()

 21%|██▏       | 3/14 [00:28<01:43,  9.44s/it]


KeyboardInterrupt: 